<a href="https://colab.research.google.com/github/LCaravaggio/NLP/blob/main/notebooks/08c_BERTClfFeatureExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos a usar BERT como feature extractor para resolver un problema de clasificación.

Una vez que obtenemos una representación vectorial de la secuencia de input, entrenamos un clasificador que podemos usar para predecir en datos nuevos.

-----------------------

Tarea: responder donde dice **PREGUNTA**

### Configuración del entorno


In [ ]:
!pip install -qU transformers datasets evaluate watermark

In [ ]:
import numpy as np
import pandas as pd
import torch
import datasets
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression

In [ ]:
%reload_ext watermark

In [ ]:
%watermark -vmp torch,transformers,datasets,evaluate,sklearn

Para usar GPU, arriba a la derecha seleccionar "Change runtime type" --> "T4 GPU"

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## Dataset

Vamos a resolver una de las tasks de GLUE:

[CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability). El objetivo es determinar is una oración es gramaticalmente correcta (1) o no (0).

In [ ]:
full_dataset = load_dataset("glue", "cola")

In [ ]:
full_dataset

In [ ]:
full_dataset["train"].features

In [ ]:
def show_random_elements(dataset, num_examples=10):
    picks = []
    for _ in range(num_examples):
        pick = np.random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = np.random.randint(0, len(dataset)-1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(full_dataset["train"], num_examples=10)

In [ ]:
print("Distribucion de clases:")
for k in full_dataset.keys():
    print(k)
    print(pd.Series(full_dataset[k]["label"]).value_counts())
    print("-"*70)


In [ ]:
full_dataset["test"][:3]

**PREGUNTA 9**: ¿por qué el set de test no tiene labels?

In [ ]:
print("Sentence length:")
for k in full_dataset.keys():
    print(k)
    largos = pd.Series(full_dataset[k]["sentence"]).str.len()
    print(np.quantile(largos, q=np.arange(0, 1.1, .1)).astype(int))
    print("-"*70)

## Tokenización y feature extraction

Vamos a cargar un modelo sin head porque solo nos interesa BERT para extraer features del texto.

In [ ]:
model_checkpoint = "distilbert-base-cased"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
print("max length:", tokenizer.model_max_length)
print("Vocab size:", tokenizer.vocab_size)

In [ ]:
# Cuando lo apliquemos, esto va a truncar segun la longitud maxima del batch
def tokenize_fn(examples):
    return tokenizer(examples["sentence"], truncation=True, padding=True, return_tensors="pt")

In [ ]:
# Aplicamos con batches iguales a cada particion (train, val, test) i.e. train es un gran batch
# Entonces cada ejemplo va a tener length = max length de su particion
# Hacemos esto porque solo vamos a hacer inferencia, no entrenar
tokenized_dataset = full_dataset.map(tokenize_fn, batched=True, batch_size=None)

In [ ]:
# NOTE para datasets grandes, suele convenir ir procesando los ejemplos on-the-fly.

In [ ]:
# ya truncamos segun la maxima longitud de train/val/test:
for split, ds in tokenized_dataset.items():
    ejemplos = ds[:3]["input_ids"]
    print(split)
    print([len(x) for x in ejemplos])

In [ ]:
# del full_dataset

In [ ]:
# automodel a secas no agrega ninguna capa (head) al modelo (body)
model = AutoModel.from_pretrained(model_checkpoint)
_ = model.to(device)

In [ ]:
# hacemos el forward pass en batches
batch_size = 10

In [ ]:
# Representamos cada input con el embedding CLS
# --> extraemos el embedding de CLS en un batch de prueba

batch_prueba = {
    "attention_mask": torch.tensor(tokenized_dataset["train"][:batch_size]["attention_mask"], device=device),
    "input_ids": torch.tensor(tokenized_dataset["train"][:batch_size]["input_ids"], device=device),
}
model.eval()
with torch.inference_mode(): # como no_grad() pero mejor https://pytorch.org/docs/stable/generated/torch.inference_mode.html
    output_prueba = model(**batch_prueba)
cls_token_output = output_prueba.last_hidden_state[:, 0]

print(output_prueba.last_hidden_state.shape)
print(cls_token_output.shape)

In [ ]:
def get_embeddings(examples):
    """Usamos embedding de CLS para representar cada secuencia
    De qué otra manera podemos extraer embeddings?
    """
    inputs = {key: torch.tensor(data, device=device) for key, data in examples.items() if key in ['input_ids', 'attention_mask']}
    with torch.inference_mode():
        output = model(**inputs).last_hidden_state[:, 0]
    return {"features": output.cpu()}

In [ ]:
model.eval()
featurized_dataset = tokenized_dataset.map(get_embeddings, batched=True, batch_size=batch_size)

In [ ]:
len(featurized_dataset["train"]["features"]), len(featurized_dataset["train"]["features"][0])

In [ ]:
# usamos arrays de numpy para entrenar/evaluar el modelo
X_train = np.array(featurized_dataset["train"]["features"])
y_train = np.array(featurized_dataset["train"]["label"])

X_val = np.array(featurized_dataset["validation"]["features"])
y_val = np.array(featurized_dataset["validation"]["label"])

X_test = np.array(featurized_dataset["test"]["features"])
y_test = np.array(featurized_dataset["test"]["label"])

**PREGUNTA 10**: ¿qué dimensión tiene cada ejemplo "vectorizado"? ¿Qué tipo de _pooling_ usamos para extraer los vectores?

## Modelo

Entrenado sobre los BERT embeddings ya extraidos.

Vamos a hacer _error analysis_ (inspeccionar los ejemplos peor puntuados por el modelo).

In [ ]:
mod = LogisticRegression(max_iter=1000)
mod.fit(X_train, y_train)

**PREGUNTA 11**: ¿qué otro modelo podríamos usar en lugar de la regresión logística? ¿qué ventajas / desventajas tiene la reg. logística?

In [ ]:
metric = evaluate.load('glue', "cola") # matthews corr coefficient

In [ ]:
scores_train = mod.predict_proba(X_train)[:, 1]
pred_train = scores_train.round() # clf con argmax (no ideal)
metric.compute(predictions=pred_train, references=y_train)

In [ ]:
scores_val = mod.predict_proba(X_val)[:, 1]
pred_val = scores_val.round()
metric.compute(predictions=pred_val, references=y_val)

In [ ]:
df_val = pd.DataFrame({"y": y_val, "score": scores_val, "idx": featurized_dataset["validation"]["idx"]})

In [ ]:
# falsos positivos más fuertes (y=0 --> no aceptable)
top_fp = df_val.query("y == 0").sort_values("score", ascending=False).head(5)
top_fp

In [ ]:
featurized_dataset["validation"].select(top_fp["idx"])["sentence"]

In [ ]:
# falsos negativos mas fuertes (y=1 --> aceptable)
top_fn = df_val.query("y == 1").sort_values("score", ascending=True).head(5)
top_fn

In [ ]:
featurized_dataset["validation"].select(top_fn["idx"])["sentence"]

## Referencias

* [Notebooks de rasbt](https://github.com/rasbt/deeplearning-models#transformers)
* [Notebooks de HuggingFace](https://huggingface.co/docs/transformers/notebooks)